In [ ]:
import pickle
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from alexnet import AlexNet

# TODO: Load traffic signs data.
training_file = "train.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
    
x_list, y_list = train['features'], train['labels']

# TODO: Split data into training and validation sets.
size_y = y_list.shape[0]
valid_size = int(size_y*0.1)
train_size = size_y-valid_size
n_chn      = x_list[0].shape[2]
n_class    = len(np.unique(y_list))
AlexNet_size = 227

print(size_y)
print(train_size)
print(valid_size)
print(train_size + valid_size)
print(n_chn)
print(n_class)

x_train = x_list[:train_size]
x_valid = x_list[train_size:]
y_train = y_list[:train_size]
y_valid = y_list[train_size:]

print("x_train", x_train.shape)
print("y_train", y_train.shape)
print("x_valid", x_valid.shape)
print("y_valid", y_valid.shape)



# TODO: Define placeholders and resize operation.
# x placeholder to be in shape of nonex227x227x3 to match the size of AlexNet
x = tf.placeholder(tf.float32, (None, 32, 32,  n_chn))
y = tf.placeholder(tf.int32, (None))
resized = tf.image.resize_images(x, (AlexNet_size, AlexNet_size))
rate = 0.5
one_hot_y  = tf.one_hot(y, n_class)
BATCH_SIZE = 64
EPOCHS = 10



# TODO: pass placeholder as first argument to `AlexNet`.
fc7 = AlexNet(resized, feature_extract=True)
# NOTE: `tf.stop_gradient` prevents the gradient from flowing backwards
# past this point, keeping the weights before and up to `fc7` frozen.
# This also makes training faster, less work to do!
fc7 = tf.stop_gradient(fc7)

# TODO: Add the final layer for traffic sign classification.
# use this shape for the weight matrix (size of fc7 output shape last element and size of n_class)
fc7shape = (fc7.get_shape().as_list()[-1], n_class)
print("fc7shape", fc7shape, fc7shape[0], fc7shape[-1], [fc7shape], np.array([fc7shape]), np.array(fc7shape))

fc8w  = tf.Variable(tf.truncated_normal([fc7shape[0], fc7shape[1]], mean = 0, stddev = 0.01))
fc8b  = tf.Variable(tf.zeros(fc7shape[1]))
logits= tf.add(tf.matmul(fc7, fc8w), fc8b)
probs = tf.nn.softmax(logits)
print("fc8w = ", fc8w.shape)
print("vd8b = ", fc8b.shape)
print("probs = ", probs)



# TODO: Define loss, training, accuracy operations.
# HINT: Look back at your traffic signs project solution, you may
# be able to reuse some the code.
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
print("cross_entropy=", cross_entropy)
loss_operation = tf.reduce_mean(cross_entropy)
print("loss_operation=", loss_operation)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)
print("training_operation=", training_operation)


#evaluation
print("tf.argmax(logits, 1)=", tf.argmax(logits, 1))
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data, keep_prob=1):
    num_examples = X_data.shape[0]
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_probs:keep_prob})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples


# TODO: Train and evaluate the feature extraction model.
#Training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = x_train.shape[0]
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        #x_strain, y_strain = shuffle(x_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = x_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x:batch_x, y:batch_y})
            
        validation_accuracy = evaluate(x_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")



39209
35289
3920
39209
3
43
x_train (35289, 32, 32, 3)
y_train (35289,)
x_valid (3920, 32, 32, 3)
y_valid (3920,)
conv1 (?, 57, 57, 96)
maxpool1 (?, 28, 28, 96)
conv2 (?, 28, 28, 256)
maxpool2 (?, 13, 13, 256)
conv3 (?, 13, 13, 384)
conv4 (?, 13, 13, 384)
conv5 (?, 13, 13, 256)
maxpool5 (?, 6, 6, 256)
fc6 (?, 4096)
fc7 (?, 4096)
fc7shape (4096, 43) 4096 43 [(4096, 43)] [[4096   43]] [4096   43]
fc8w =  (4096, 43)
vd8b =  (43,)
probs =  Tensor("Softmax_5:0", shape=(?, 43), dtype=float32)
cross_entropy= Tensor("Reshape_53:0", shape=(?,), dtype=float32)
loss_operation= Tensor("Mean_10:0", shape=(), dtype=float32)
training_operation= name: "Adam_5"
op: "NoOp"
input: "^Adam_5/update_Variable_94/ApplyAdam"
input: "^Adam_5/update_Variable_95/ApplyAdam"
input: "^Adam_5/Assign"
input: "^Adam_5/Assign_1"

tf.argmax(logits, 1)= Tensor("ArgMax_15:0", shape=(?,), dtype=int64)
Training...

